# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおいて、異なる大規模言語モデル（LLM）の生成した応答の中から、どちらがより好まれるかを予測するための問題に取り組んでいます。Notebookでは、主に以下の流れで問題解決が行われています。

### 問題の背景
コンペティションでは、ユーザーが2つの匿名化されたチャットボットの応答から好みのものを選ぶタスクに基づいて、どちらの応答が選ばれるかを予測するモデルの構築が求められています。これに対して、いくつかのアプローチとモデルが提案されています。

### 使用している手法とライブラリ
1. **ライブラリのインストールとインポート**: 
   - `transformers`, `peft`, `accelerate`, `bitsandbytes` などのライブラリを利用して、深層学習と自然言語処理のタスクを行います。
   - `torch` (PyTorch)や`sklearn` (Scikit-learn)なども使用されています。

2. **トークナイザーの設定とデータ処理**:
   - 入力データの前処理として、テキストをトークナイズし、モデルが扱いやすい形式に変換します。
   - `AutoTokenizer`や`GemmaTokenizerFast`を使用して、テキストデータをトークナイズし、アテンションマスクを生成します。

3. **モデルの設定と学習**:
   - `LlamaForSequenceClassification`や`Gemma2ForSequenceClassification`などの事前訓練済みモデルをロードし、それぞれ異なるGPUで推論を行う設定をしています。
   - LoRA (Low-Rank Adaptation)技術を使用することで、モデルの重みを効率的に調整し、推論の精度を上げています。

4. **推論の実行**:
   - スレッドを利用して並列処理を行い、モデルの出力から各応答の勝率を計算します。
   - 結果はDataFrameに格納され、最終的な予測を得るために統合されます。

5. **最終予測の算出**:
   - LlamaモデルとLightGBMを組み合わせたアンサンブル学習により、より robust な予測を実現しています。
   - `submission.csv`として結果を出力するためのデータフレームを準備し、最終的な提出用ファイルが生成されています。

このノートブックは、LLMを活用した人間の好みの予測という複雑なタスクに対して、最先端の技術を利用し、モデルのパフォーマンスを最大限に引き出すための包括的なアプローチを採用しています。

---


# 用語概説 
以下に、Jupyter Notebookの内容において、初心者がつまずきそうな専門用語や概念の解説を提供します。特にマイナーなものや実務経験がないと馴染みのないものに焦点を合わせています。

### 専門用語解説

1. **PEFT（Parameter-Efficient Fine-Tuning）**:
   - 事前学習済みモデルのファインチューニングを効率的に行うための手法。モデル全体を再学習することなく、一部のパラメータのみを調整することで、計算リソースを節約しやすくなります。

2. **LoRA（Low-Rank Adaptation）**:
   - PEFTの一種で、低ランク行列を用いて事前学習済みモデルの重みを適応的に調整します。少ないパラメータ変更で性能を向上させることを目的とし、より効率的にトレーニングが可能になります。

3. **BitsAndBytes**:
   - 大規模言語モデルを効率的に運用するためのライブラリで、重みの圧縮や量子化技術を提供します。特に8ビット計算を用いることで、メモリ使用量を削減しつつ推論速度を向上させることを目的としています。

4. **AutoMixedPrecision (自動混合精度)**:
   - モデルの学習や推論時に、計算精度を自動的に調整する機能です。特に、16ビットと32ビットの浮動小数点数を組み合わせて使用することで、メモリの節約と計算の高速化を実現します。

5. **アテンションマスク**:
   - トランスフォーマーモデルにおいて、どのトークンを参照するべきかを示すためのマスクです。特に、パディングされた部分（無効なトークン）は無視されるように設定されます。

6. **トークナイザー**:
   - 自然言語処理において、文書を単語やサブワードに分割するためのコンポーネントです。トークナイザーは、モデルが理解できる形式でデータを整形するために使用されます。

7. **パディング（Padding）**:
   - シーケンスデータの長さを揃えるために、短いシーケンスの末尾に特定のトークン（通常はゼロトークン）を追加する処理です。モデルは固定長の入力を必要とするため、パディングが役立ちます。

8. **GitOpsやCI/CDパイプライン**:
   - 通常はソフトウェア開発に関連する用語ですが、機械学習モデルのデプロイや管理においても適用されることがあります。例えば、MLのモデルの更新フローを自動化することで、迅速かつ安定したリリースが可能になります。

9. **シャーディング（Sharding）**:
   - 大きなモデルを複数のデバイスに分けて並行処理する手法です。GPUのメモリ制限を克服しつつ、モデルの推論やトレーニングを効率化します。

10. **ダイナミックバッチング（Dynamic Batching）**:
   - リアルタイム推論中にリクエストをまとめて処理することで、より効率的に計算を行う手法です。リクエストの長さや数によってバッチサイズを変更します。

これらの用語や概念は、特に機械学習・深層学習初心者が理解する際に、注意が必要です。理解を深めるためには、関連する文献やオンラインリソースを活用することも良いでしょう。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install transformers peft accelerate bitsandbytes \-U --no-index --find-links /kaggle/input/lmsys-wheel-files
```

</div>
<div class="column-right">

# 日本語訳

```python
# 必要なライブラリをインストールする
# transformers、peft、accelerate、bitsandbytesを最新バージョンでインストールします。
!pip install transformers peft accelerate bitsandbytes \-U --no-index --find-links /kaggle/input/lmsys-wheel-files
```

</div>
</details>

In [ ]:
# 必要なライブラリをインストールする
# transformers、peft、accelerate、bitsandbytesを最新バージョンでインストールします。
!pip install transformers peft accelerate bitsandbytes \-U --no-index --find-links /kaggle/input/lmsys-wheel-files

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
<div class="column-right">

# 日本語訳

```python
# bitsandbytesライブラリをインストールする（-qで出力を抑制）
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
# transformersライブラリをインストールする（-qで出力を抑制）
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
# tokenizersライブラリをインストールする（-qで出力を抑制）
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
# peftライブラリをインストールする（-qで出力を抑制）
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
</details>

In [ ]:
# bitsandbytesライブラリをインストールする（-qで出力を抑制）
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
# transformersライブラリをインストールする（-qで出力を抑制）
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
# tokenizersライブラリをインストールする（-qで出力を抑制）
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
# peftライブラリをインストールする（-qで出力を抑制）
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# PART 1 (LLama 3 8b Chat)

</div>
<div class="column-right">

# 日本語訳

# パート 1 (LLama 3 8b チャット)

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import torch
import sklearn
import numpy as np
import pandas as pd
import time

from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from torch.cuda.amp import autocast
from threading import Thread

import gc
import os
import io
import time
import json
import random
import pickle
import zipfile
import datetime
import matplotlib.pyplot as plt
from IPython.display import display
from collections import Counter
from collections import defaultdict
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import log_loss
import tokenizers
```

</div>
<div class="column-right">

# 日本語訳

```python
# 必要なライブラリをインポートする
import torch  # PyTorchライブラリをインポート
import sklearn  # scikit-learnライブラリをインポート
import numpy as np  # NumPyライブラリをインポート
import pandas as pd  # Pandasライブラリをインポート
import time  # 時間関連の機能を使用するためにtimeモジュールをインポート

# Transformersライブラリから必要なモデルやトークナイザーをインポート
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
# PEFTライブラリから必要なモジュールをインポート
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
# 自動混合精度を使用するためにautocastをインポート
from torch.cuda.amp import autocast
# スレッドを使用するためにThreadをインポート
from threading import Thread

# ガベージコレクションを制御するためにgcモジュールをインポート
import gc
import os  # オペレーティングシステム関連の機能を使用するためにosモジュールをインポート
import io  # 入出力操作を行うためにioモジュールをインポート
import time  # 時間関連の機能を使用するためにtimeモジュールを再度インポート
import json  # JSONデータを扱うためにjsonモジュールをインポート
import random  # ランダム数生成のためにrandomモジュールをインポート
import pickle  # Pythonオブジェクトのシリアル化のためにpickleモジュールをインポート
import zipfile  # ZIPファイルを扱うためにzipfileモジュールをインポート
import datetime  # 日付と時刻を扱うためにdatetimeモジュールをインポート
import matplotlib.pyplot as plt  # データの可視化のためにMatplotlibをインポート
from IPython.display import display  # IPython環境での出力を行うためにdisplay関数をインポート
from collections import Counter  # 要素のカウントのためのCounterクラスをインポート
from collections import defaultdict  # デフォルト値を持つ辞書を作成するためのdefaultdictクラスをインポート
import torch  # PyTorchライブラリを再度インポート
from torch import nn  # ニューラルネットワーク関連モジュールをインポート
import torch.nn.functional as F  # ニューラルネットワークの関数をインポート
import pytorch_lightning as pl  # PyTorch Lightningをインポート
from torch.utils.data import Dataset, DataLoader  # データセットとデータローダーをインポート
from sklearn.metrics import log_loss  # ログ損失の計算のためにインポート
import tokenizers  # トークナイザーを扱うためにtokenizersライブラリをインポート
```

</div>
</details>

In [ ]:
# 必要なライブラリをインポートする
import torch  # PyTorchライブラリをインポート
import sklearn  # scikit-learnライブラリをインポート
import numpy as np  # NumPyライブラリをインポート
import pandas as pd  # Pandasライブラリをインポート
import time  # 時間関連の機能を使用するためにtimeモジュールをインポート

# Transformersライブラリから必要なモデルやトークナイザーをインポート
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
# PEFTライブラリから必要なモジュールをインポート
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
# 自動混合精度を使用するためにautocastをインポート
from torch.cuda.amp import autocast
# スレッドを使用するためにThreadをインポート
from threading import Thread

# ガベージコレクションを制御するためにgcモジュールをインポート
import gc
import os  # オペレーティングシステム関連の機能を使用するためにosモジュールをインポート
import io  # 入出力操作を行うためにioモジュールをインポート
import time  # 時間関連の機能を使用するためにtimeモジュールを再度インポート
import json  # JSONデータを扱うためにjsonモジュールをインポート
import random  # ランダム数生成のためにrandomモジュールをインポート
import pickle  # Pythonオブジェクトのシリアル化のためにpickleモジュールをインポート
import zipfile  # ZIPファイルを扱うためにzipfileモジュールをインポート
import datetime  # 日付と時刻を扱うためにdatetimeモジュールをインポート
import matplotlib.pyplot as plt  # データの可視化のためにMatplotlibをインポート
from IPython.display import display  # IPython環境での出力を行うためにdisplay関数をインポート
from collections import Counter  # 要素のカウントのためのCounterクラスをインポート
from collections import defaultdict  # デフォルト値を持つ辞書を作成するためのdefaultdictクラスをインポート
import torch  # PyTorchライブラリを再度インポート
from torch import nn  # ニューラルネットワーク関連モジュールをインポート
import torch.nn.functional as F  # ニューラルネットワークの関数をインポート
import pytorch_lightning as pl  # PyTorch Lightningをインポート
from torch.utils.data import Dataset, DataLoader  # データセットとデータローダーをインポート
from sklearn.metrics import log_loss  # ログ損失の計算のためにインポート
import tokenizers  # トークナイザーを扱うためにtokenizersライブラリをインポート

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel
```

</div>
<div class="column-right">

# 日本語訳

```python
# データクラスを使用するためにdataclassをインポート
from dataclasses import dataclass
# スレッドプールを使用するためにThreadPoolExecutorをインポート
from concurrent.futures import ThreadPoolExecutor

# TransformersライブラリからGemmaモデルとトークナイザーをインポート
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
# トークナイザーの警告を抑制しながらパディングを行うためのデータコレーターをインポート
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
# PEFTライブラリからPeftModelをインポート
from peft import PeftModel
```

</div>
</details>

In [ ]:
# データクラスを使用するためにdataclassをインポート
from dataclasses import dataclass
# スレッドプールを使用するためにThreadPoolExecutorをインポート
from concurrent.futures import ThreadPoolExecutor

# TransformersライブラリからGemmaモデルとトークナイザーをインポート
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
# トークナイザーの警告を抑制しながらパディングを行うためのデータコレーターをインポート
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
# PEFTライブラリからPeftModelをインポート
from peft import PeftModel

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

if (not torch.cuda.is_available()): print("Sorry - GPU required!")

MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'
MAX_LENGTH = 1284
BATCH_SIZE = 8
DEVICE = torch.device("cuda")    
```

</div>
<div class="column-right">

# 日本語訳

```python
# メモリ効率の良いシステムダイナミクスプログラミング（SDP）を無効にする
torch.backends.cuda.enable_mem_efficient_sdp(False)
# フラッシュSDPを無効にする
torch.backends.cuda.enable_flash_sdp(False)

# GPUが利用可能でない場合の警告メッセージ
if (not torch.cuda.is_available()): 
    print("申し訳ありませんが、GPUが必要です！")

# モデルと重みのパスを設定
MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'

# モデルの最大入力長とバッチサイズを設定
MAX_LENGTH = 1284  # 最大入力長を1284に設定
BATCH_SIZE = 8  # バッチサイズを8に設定

# 計算デバイスを指定（GPUを使用）
DEVICE = torch.device("cuda")
```

</div>
</details>

In [ ]:
# メモリ効率の良いシステムダイナミクスプログラミング（SDP）を無効にする
torch.backends.cuda.enable_mem_efficient_sdp(False)
# フラッシュSDPを無効にする
torch.backends.cuda.enable_flash_sdp(False)

# GPUが利用可能でない場合の警告メッセージ
if (not torch.cuda.is_available()): 
    print("申し訳ありませんが、GPUが必要です！")

# モデルと重みのパスを設定
MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'

# モデルの最大入力長とバッチサイズを設定
MAX_LENGTH = 1284  # 最大入力長を1284に設定
BATCH_SIZE = 8  # バッチサイズを8に設定

# 計算デバイスを指定（GPUを使用）
DEVICE = torch.device("cuda")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータセットをCSVファイルから読み込む
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
# 提出サンプルをCSVファイルから読み込む
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')
```

</div>
</details>

In [ ]:
# テストデータセットをCSVファイルから読み込む
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
# 提出サンプルをCSVファイルから読み込む
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# concatenate strings in list
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

display(sample_sub)
display(test.head(5))

# Prepare text for model
test['text'] = 'User prompt: ' + test['prompt'] +  '\n\nModel A :\n' + test['response_a'] +'\n\n--------\n\nModel B:\n'  + test['response_b']
print(test['text'][0])
```

</div>
<div class="column-right">

# 日本語訳

```python
# リスト内の文字列を結合する関数を定義
def process(input_str):
    # 入力文字列の両端のブラケットを削除
    stripped_str = input_str.strip('[]')
    # カンマで区切った部分をリストに分割し、余分な引用符を削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    # リストの要素を空白で結合して一つの文字列にする
    return  ' '.join(sentences)

# データフレームの各列に対してprocess関数を適用
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

# 提出サンプルとテストデータの最初の5行を表示
display(sample_sub)
display(test.head(5))

# モデル用にテキストを準備
test['text'] = 'ユーザープロンプト: ' + test['prompt'] +  '\n\nモデル A:\n' + test['response_a'] +'\n\n--------\n\nモデル B:\n'  + test['response_b']
# 最初のテキストを表示
print(test['text'][0])
```

</div>
</details>

In [ ]:
# リスト内の文字列を結合する関数を定義
def process(input_str):
    # 入力文字列の両端のブラケットを削除
    stripped_str = input_str.strip('[]')
    # カンマで区切った部分をリストに分割し、余分な引用符を削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    # リストの要素を空白で結合して一つの文字列にする
    return  ' '.join(sentences)

# データフレームの各列に対してprocess関数を適用
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

# 提出サンプルとテストデータの最初の5行を表示
display(sample_sub)
display(test.head(5))

# モデル用にテキストを準備
test['text'] = 'ユーザープロンプト: ' + test['prompt'] +  '\n\nモデル A:\n' + test['response_a'] +'\n\n--------\n\nモデル B:\n'  + test['response_b']
# 最初のテキストを表示
print(test['text'][0])

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Tokenize

</div>
<div class="column-right">

# 日本語訳

## トークナイズ

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# Move tensors to CPU and convert them to lists
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu
data['ATTENTION_MASKS'] = attention_masks_cpu
data[:2]
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイザーを指定したパスからロードする
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# テキストをトークナイズし、パディング、トランケーションを行う
tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

# 入力IDとアテンションマスクを指定したデバイスに移動し、データ型を整数32ビットに指定
INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# テンソルをCPUに移動させ、リストに変換
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

# データフレームを作成し、入力IDとアテンションマスクを格納
data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu
data['ATTENTION_MASKS'] = attention_masks_cpu
# 最初の2行を表示
data[:2]
```

</div>
</details>

In [ ]:
# トークナイザーを指定したパスからロードする
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# テキストをトークナイズし、パディング、トランケーションを行う
tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

# 入力IDとアテンションマスクを指定したデバイスに移動し、データ型を整数32ビットに指定
INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# テンソルをCPUに移動させ、リストに変換
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

# データフレームを作成し、入力IDとアテンションマスクを格納
data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu
data['ATTENTION_MASKS'] = attention_masks_cpu
# 最初の2行を表示
data[:2]

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Load model 
> We load 1 model on each gpu.  

</div>
<div class="column-right">

# 日本語訳

## モデルのロード
> 各GPUに1つのモデルをロードします。  

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# BitsAndBytes configuration
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=False)

# Load base model on GPU 0
device0 = torch.device('cuda:0')

base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:0')
base_model_0.config.pad_token_id = tokenizer.pad_token_id
```

</div>
<div class="column-right">

# 日本語訳

```python
# BitsAndBytesの設定を行う
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットでのロードを指定
    bnb_8bit_compute_dtype=torch.float16,  # 計算に使用する8ビットのデータ型をfloat16に指定
    bnb_8bit_use_double_quant=False  # ダブル量子化を使用しない場合はFalse
)

# GPU 0にベースモデルをロード
device0 = torch.device('cuda:0')

# LlamaForSequenceClassificationモデルを指定した設定でロード
base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # ラベルの数を3に指定
    torch_dtype=torch.float16,  # モデルパラメータのデータ型をfloat16に指定
    quantization_config=bnb_config,  # 量子化設定を適用
    device_map='cuda:0'  # モデルをGPU 0にマップ
)
# パディングトークンIDを設定
base_model_0.config.pad_token_id = tokenizer.pad_token_id
```

</div>
</details>

In [ ]:
# BitsAndBytesの設定を行う
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットでのロードを指定
    bnb_8bit_compute_dtype=torch.float16,  # 計算に使用する8ビットのデータ型をfloat16に指定
    bnb_8bit_use_double_quant=False  # ダブル量子化を使用しない場合はFalse
)

# GPU 0にベースモデルをロード
device0 = torch.device('cuda:0')

# LlamaForSequenceClassificationモデルを指定した設定でロード
base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # ラベルの数を3に指定
    torch_dtype=torch.float16,  # モデルパラメータのデータ型をfloat16に指定
    quantization_config=bnb_config,  # 量子化設定を適用
    device_map='cuda:0'  # モデルをGPU 0にマップ
)
# パディングトークンIDを設定
base_model_0.config.pad_token_id = tokenizer.pad_token_id

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Load base model on GPU 1
device1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:1')
base_model_1.config.pad_token_id = tokenizer.pad_token_id
```

</div>
<div class="column-right">

# 日本語訳

```python
# GPU 1にベースモデルをロード
device1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # ラベルの数を3に指定
    torch_dtype=torch.float16,  # モデルパラメータのデータ型をfloat16に指定
    quantization_config=bnb_config,  # 量子化設定を適用
    device_map='cuda:1'  # モデルをGPU 1にマップ
)
# パディングトークンIDを設定
base_model_1.config.pad_token_id = tokenizer.pad_token_id
```

</div>
</details>

In [ ]:
# GPU 1にベースモデルをロード
device1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # ラベルの数を3に指定
    torch_dtype=torch.float16,  # モデルパラメータのデータ型をfloat16に指定
    quantization_config=bnb_config,  # 量子化設定を適用
    device_map='cuda:1'  # モデルをGPU 1にマップ
)
# パディングトークンIDを設定
base_model_1.config.pad_token_id = tokenizer.pad_token_id

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# LoRa configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.10,
    bias='none',
    inference_mode=True,
    task_type=TaskType.SEQ_CLS,
    target_modules=['o_proj', 'v_proj'])
```

</div>
<div class="column-right">

# 日本語訳

```python
# LoRa（Low-Rank Adaptation）の設定を行う
peft_config = LoraConfig(
    r=16,  # ランクを16に設定
    lora_alpha=32,  # LoRaのスケーリングファクターを32に設定
    lora_dropout=0.10,  # LoRaのドロップアウト率を10%に設定
    bias='none',  # バイアスの設定をなしに指定
    inference_mode=True,  # 推論モードを有効にする
    task_type=TaskType.SEQ_CLS,  # タスクタイプを列分類に設定
    target_modules=['o_proj', 'v_proj']  # 対象モジュールを指定
)
```

</div>
</details>

In [ ]:
# LoRa（Low-Rank Adaptation）の設定を行う
peft_config = LoraConfig(
    r=16,  # ランクを16に設定
    lora_alpha=32,  # LoRaのスケーリングファクターを32に設定
    lora_dropout=0.10,  # LoRaのドロップアウト率を10%に設定
    bias='none',  # バイアスの設定をなしに指定
    inference_mode=True,  # 推論モードを有効にする
    task_type=TaskType.SEQ_CLS,  # タスクタイプを列分類に設定
    target_modules=['o_proj', 'v_proj']  # 対象モジュールを指定
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Get peft
model_0 = get_peft_model(base_model_0, peft_config).to(device0) 
#Load weights
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_0.eval()

model_1 = get_peft_model(base_model_1, peft_config).to(device1)
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_1.eval()

#Trainable Parameters
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()
```

</div>
<div class="column-right">

# 日本語訳

```python
# PEFTモデルを取得し、GPU 0に配置
model_0 = get_peft_model(base_model_0, peft_config).to(device0)
# 重みをロード
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # 重みを指定したパスから読み込み
model_0.eval()  # 評価モードに設定

# PEFTモデルを取得し、GPU 1に配置
model_1 = get_peft_model(base_model_1, peft_config).to(device1)
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # 重みを指定したパスから読み込み
model_1.eval()  # 評価モードに設定

# 学習可能なパラメータを表示
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()
```

</div>
</details>

In [ ]:
# PEFTモデルを取得し、GPU 0に配置
model_0 = get_peft_model(base_model_0, peft_config).to(device0)
# 重みをロード
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # 重みを指定したパスから読み込み
model_0.eval()  # 評価モードに設定

# PEFTモデルを取得し、GPU 1に配置
model_1 = get_peft_model(base_model_1, peft_config).to(device1)
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # 重みを指定したパスから読み込み
model_1.eval()  # 評価モードに設定

# 学習可能なパラメータを表示
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
gc.collect()
```

</div>
<div class="column-right">

# 日本語訳

```python
# ガベージコレクションを実行して、使用されていないメモリを解放
gc.collect()
```

</div>
</details>

In [ ]:
# ガベージコレクションを実行して、使用されていないメモリを解放
gc.collect()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Inference

</div>
<div class="column-right">

# 日本語訳

## 推論

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def inference(df, model, device, batch_size=BATCH_SIZE):
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)
    
    generated_class_a = []
    generated_class_b = []
    generated_class_c = []

    model.eval()
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch_input_ids = input_ids[start_idx:end_idx].to(device)
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)
        
        with torch.no_grad():
            with autocast():
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask
                )
        
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        
        generated_class_a.extend(probabilities[:, 0])
        generated_class_b.extend(probabilities[:, 1])
        generated_class_c.extend(probabilities[:, 2])
    
    df['winner_model_a'] = generated_class_a
    df['winner_model_b'] = generated_class_b
    df['winner_tie'] = generated_class_c

    torch.cuda.empty_cache()  

    return df
```

</div>
<div class="column-right">

# 日本語訳

```python
# 推論を行う関数を定義
def inference(df, model, device, batch_size=BATCH_SIZE):
    # データフレームから入力IDとアテンションマスクを取得
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)
    
    # 各クラスの生成された確率を格納するリストを初期化
    generated_class_a = []
    generated_class_b = []
    generated_class_c = []

    model.eval()  # モデルを評価モードに設定
    
    # データをバッチ処理で推論
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終わりのインデックスを計算
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # 現在のバッチの入力IDをデバイスに移動
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # 現在のバッチのアテンションマスクをデバイスに移動
        
        with torch.no_grad():  # 勾配計算を無効にする
            with autocast():  # 自動混合精度を使用
                outputs = model(
                    input_ids=batch_input_ids,  # モデルに入力IDを渡す
                    attention_mask=batch_attention_mask  # モデルにアテンションマスクを渡す
                )
        
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()  # 出力のロジットをソフトマックス関数で確率に変換
        
        # 各クラスの確率をリストに追加
        generated_class_a.extend(probabilities[:, 0])  # クラスAの確率を追加
        generated_class_b.extend(probabilities[:, 1])  # クラスBの確率を追加
        generated_class_c.extend(probabilities[:, 2])  # 引き分けの確率を追加
    
    # データフレームに生成された確率を追加
    df['winner_model_a'] = generated_class_a
    df['winner_model_b'] = generated_class_b
    df['winner_tie'] = generated_class_c

    torch.cuda.empty_cache()  # GPUのキャッシュメモリを解放

    return df  # 推論結果が含まれたデータフレームを返す
```

</div>
</details>

In [ ]:
# 推論を行う関数を定義
def inference(df, model, device, batch_size=BATCH_SIZE):
    # データフレームから入力IDとアテンションマスクを取得
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)
    
    # 各クラスの生成された確率を格納するリストを初期化
    generated_class_a = []
    generated_class_b = []
    generated_class_c = []

    model.eval()  # モデルを評価モードに設定
    
    # データをバッチ処理で推論
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終わりのインデックスを計算
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # 現在のバッチの入力IDをデバイスに移動
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # 現在のバッチのアテンションマスクをデバイスに移動
        
        with torch.no_grad():  # 勾配計算を無効にする
            with autocast():  # 自動混合精度を使用
                outputs = model(
                    input_ids=batch_input_ids,  # モデルに入力IDを渡す
                    attention_mask=batch_attention_mask  # モデルにアテンションマスクを渡す
                )
        
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()  # 出力のロジットをソフトマックス関数で確率に変換
        
        # 各クラスの確率をリストに追加
        generated_class_a.extend(probabilities[:, 0])  # クラスAの確率を追加
        generated_class_b.extend(probabilities[:, 1])  # クラスBの確率を追加
        generated_class_c.extend(probabilities[:, 2])  # 引き分けの確率を追加
    
    # データフレームに生成された確率を追加
    df['winner_model_a'] = generated_class_a
    df['winner_model_b'] = generated_class_b
    df['winner_tie'] = generated_class_c

    torch.cuda.empty_cache()  # GPUのキャッシュメモリを解放

    return df  # 推論結果が含まれたデータフレームを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

N_SAMPLES = len(data)

# Split the data into two subsets
half = round(N_SAMPLES / 2)
sub1 = data.iloc[0:half].copy()
sub2 = data.iloc[half:N_SAMPLES].copy()

# Function to run inference in a thread
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)

# Dictionary to store results from threads
results = {}
```

</div>
<div class="column-right">

# 日本語訳

```python
# 計測開始時刻を記録
st = time.time()

N_SAMPLES = len(data)  # サンプルの総数を取得

# データを2つのサブセットに分割
half = round(N_SAMPLES / 2)  # サンプル数の半分を計算
sub1 = data.iloc[0:half].copy()  # 最初の半分をコピー
sub2 = data.iloc[half:N_SAMPLES].copy()  # 残りの半分をコピー

# スレッドで推論を実行する関数を定義
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)  # 指定したインデックスに推論結果を格納

# スレッドからの結果を格納する辞書を初期化
results = {}
```

</div>
</details>

In [ ]:
# 計測開始時刻を記録
st = time.time()

N_SAMPLES = len(data)  # サンプルの総数を取得

# データを2つのサブセットに分割
half = round(N_SAMPLES / 2)  # サンプル数の半分を計算
sub1 = data.iloc[0:half].copy()  # 最初の半分をコピー
sub2 = data.iloc[half:N_SAMPLES].copy()  # 残りの半分をコピー

# スレッドで推論を実行する関数を定義
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)  # 指定したインデックスに推論結果を格納

# スレッドからの結果を格納する辞書を初期化
results = {}

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# start threads
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))

t0.start()
t1.start()

# Wait for all threads to finish
t0.join()
t1.join()

# Combine results back into the original DataFrame
data = pd.concat([results[0], results[1]], axis=0)

print(f"Processing complete. Total time: {time.time() - st}")

TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

sample_sub[TARGETS] = data[TARGETS]
```

</div>
<div class="column-right">

# 日本語訳

```python
# スレッドを開始
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # サブセット1の推論を実行するスレッド
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # サブセット2の推論を実行するスレッド

t0.start()  # スレッド0を開始
t1.start()  # スレッド1を開始

# すべてのスレッドが終了するのを待機
t0.join()
t1.join()

# 推論結果を元のデータフレームに統合
data = pd.concat([results[0], results[1]], axis=0)

# 処理が完了したことを出力
print(f"処理が完了しました。総時間: {time.time() - st}")

# 提出ファイル用のターゲットを設定
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

# サンプル提出データフレームに結果を追加
sample_sub[TARGETS] = data[TARGETS]
```

</div>
</details>

In [ ]:
# スレッドを開始
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # サブセット1の推論を実行するスレッド
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # サブセット2の推論を実行するスレッド

t0.start()  # スレッド0を開始
t1.start()  # スレッド1を開始

# すべてのスレッドが終了するのを待機
t0.join()
t1.join()

# 推論結果を元のデータフレームに統合
data = pd.concat([results[0], results[1]], axis=0)

# 処理が完了したことを出力
print(f"処理が完了しました。総時間: {time.time() - st}")

# 提出ファイル用のターゲットを設定
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

# サンプル提出データフレームに結果を追加
sample_sub[TARGETS] = data[TARGETS]

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
llama_preds = data[TARGETS].values
```

</div>
<div class="column-right">

# 日本語訳

```python
# ターゲットの結果をNumPy配列に変換
llama_preds = data[TARGETS].values  # 予測結果を取得し、NumPy配列に格納
```

</div>
</details>

In [ ]:
# ターゲットの結果をNumPy配列に変換
llama_preds = data[TARGETS].values  # 予測結果を取得し、NumPy配列に格納

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## LGBM + tfidf

</div>
<div class="column-right">

# 日本語訳

## LGBM + TF-IDF

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
TAG = 'lmsys-chatbot-arena'

import os
RUNPOD = os.path.exists('/workspace/')
KAGGLE = not RUNPOD
if KAGGLE: print('kaggle')
```

</div>
<div class="column-right">

# 日本語訳

```python
# コンペティションのタグを設定
TAG = 'lmsys-chatbot-arena'

import os  # osモジュールをインポート
# RUNPOD 環境であるかをチェック
RUNPOD = os.path.exists('/workspace/')  
# Kaggle 環境かどうかを設定
KAGGLE = not RUNPOD  
if KAGGLE: 
    print('kaggle')  # Kaggle環境の場合、メッセージを出力
```

</div>
</details>

In [ ]:
# コンペティションのタグを設定
TAG = 'lmsys-chatbot-arena'

import os  # osモジュールをインポート
# RUNPOD 環境であるかをチェック
RUNPOD = os.path.exists('/workspace/')  
# Kaggle 環境かどうかを設定
KAGGLE = not RUNPOD  
if KAGGLE: 
    print('kaggle')  # Kaggle環境の場合、メッセージを出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
try:
    import pandas as pd
except:
    !pip install -q kaggle
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm 
    !pip install -q protobuf 
    !pip install -q numba
```

</div>
<div class="column-right">

# 日本語訳

```python
# pandasライブラリをインポートを試みる
try:
    import pandas as pd
except:
    # pandasがインポートできなかった場合は、必要なライブラリをインストール
    !pip install -q kaggle  # Kaggleライブラリをインストール
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm  # データ処理と可視化に必要なライブラリをインストール
    !pip install -q protobuf  # protobufライブラリをインストール
    !pip install -q numba  # numbaライブラリをインストール
```

</div>
</details>

In [ ]:
# pandasライブラリをインポートを試みる
try:
    import pandas as pd
except:
    # pandasがインポートできなかった場合は、必要なライブラリをインストール
    !pip install -q kaggle  # Kaggleライブラリをインストール
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm  # データ処理と可視化に必要なライブラリをインストール
    !pip install -q protobuf  # protobufライブラリをインストール
    !pip install -q numba  # numbaライブラリをインストール

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)

import os

if RUNPOD:
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json
        !chmod 600 /root/.kaggle/kaggle.json

    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/ 
        
    if not os.path.exists('/data/'):
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')    
```

</div>
<div class="column-right">

# 日本語訳

```python
# データのパスを設定
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)

import os  # osモジュールをインポート

# RUNPOD環境での処理
if RUNPOD:
    # Kaggleの認証ファイルが存在しない場合
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle  # kaggleディレクトリを作成
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json  # kaggle.jsonをコピー
        !chmod 600 /root/.kaggle/kaggle.json  # ファイルのアクセス権を設定

    # Kaggleのデータセットが存在しない場合
    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/  # Kaggleからデータセットをダウンロード
        
    # データがまだ存在しない場合
    if not os.path.exists('/data/'):
        import zipfile  # zipfileモジュールをインポート
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')  # ZIPファイルを解凍
```

</div>
</details>

In [ ]:
# データのパスを設定
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)

import os  # osモジュールをインポート

# RUNPOD環境での処理
if RUNPOD:
    # Kaggleの認証ファイルが存在しない場合
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle  # kaggleディレクトリを作成
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json  # kaggle.jsonをコピー
        !chmod 600 /root/.kaggle/kaggle.json  # ファイルのアクセス権を設定

    # Kaggleのデータセットが存在しない場合
    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/  # Kaggleからデータセットをダウンロード
        
    # データがまだ存在しない場合
    if not os.path.exists('/data/'):
        import zipfile  # zipfileモジュールをインポート
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')  # ZIPファイルを解凍

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
INPUT_PATH = '/kaggle/input/'  
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'
# MODEL_PATH = MODEL_PATH if not KAGGLE else ''#MODEL_PATH + os.listdir(MODEL_PATH)[0] + '/'
print(MODEL_PATH)

CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'
SAVE_PATH = MODEL_PATH if not KAGGLE else ''
```

</div>
<div class="column-right">

# 日本語訳

```python
# 入力パス、モデルパス、ロジットパスを設定
INPUT_PATH = '/kaggle/input/'  
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'

# Kaggle環境の場合のモデルパスの設定
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'

# モデルパスを出力
print(MODEL_PATH)

# コードパスと保存パスの設定
CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'
SAVE_PATH = MODEL_PATH if not KAGGLE else ''
```

</div>
</details>

In [ ]:
# 入力パス、モデルパス、ロジットパスを設定
INPUT_PATH = '/kaggle/input/'  
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'

# Kaggle環境の場合のモデルパスの設定
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'

# モデルパスを出力
print(MODEL_PATH)

# コードパスと保存パスの設定
CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'
SAVE_PATH = MODEL_PATH if not KAGGLE else ''

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイザーの並列処理を無効にするための環境変数を設定
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
```

</div>
</details>

In [ ]:
# トークナイザーの並列処理を無効にするための環境変数を設定
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train = pd.read_csv(open(DATA + 'train.csv', 'r'))
test = pd.read_csv(open(DATA + 'test.csv', 'r'))
sample = pd.read_csv(DATA + 'sample_submission.csv')

print(len(train), len(test))
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータ、テストデータ、サンプル提出ファイルをCSVから読み込む
train = pd.read_csv(open(DATA + 'train.csv', 'r'))  # トレーニングデータを読み込む
test = pd.read_csv(open(DATA + 'test.csv', 'r'))    # テストデータを読み込む
sample = pd.read_csv(DATA + 'sample_submission.csv')  # サンプル提出データを読み込む

# トレーニングデータとテストデータのサンプル数を出力
print(len(train), len(test))
```

</div>
</details>

In [ ]:
# トレーニングデータ、テストデータ、サンプル提出ファイルをCSVから読み込む
train = pd.read_csv(open(DATA + 'train.csv', 'r'))  # トレーニングデータを読み込む
test = pd.read_csv(open(DATA + 'test.csv', 'r'))    # テストデータを読み込む
sample = pd.read_csv(DATA + 'sample_submission.csv')  # サンプル提出データを読み込む

# トレーニングデータとテストデータのサンプル数を出力
print(len(train), len(test))

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
params = {}
if False:#len(test) < 10: 
    pass;
    params['subsample'] = 30
else:
    # params['subsample'] = 2
    params['fold'] = -1


params['n_epochs'] = 1
params['n_lgb'] = 1
params['model'] = 'microsoft/deberta-v3-small'
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルのパラメータを設定
params = {}
if False:  # テストデータの長さが10未満でない限り
    pass;
    params['subsample'] = 30  # サブサンプルサイズを30に設定
else:
    # params['subsample'] = 2  # サブサンプルサイズを2に設定（コメントアウト）
    params['fold'] = -1  # フォールド設定を-1に設定

# エポック数とLightGBMのモデル数を設定
params['n_epochs'] = 1  # エポック数を1に設定
params['n_lgb'] = 1  # 使用するLightGBMのモデル数を1に設定
params['model'] = 'microsoft/deberta-v3-small'  # 使用するモデルを指定
```

</div>
</details>

In [ ]:
# モデルのパラメータを設定
params = {}
if False:  # テストデータの長さが10未満でない限り
    pass;
    params['subsample'] = 30  # サブサンプルサイズを30に設定
else:
    # params['subsample'] = 2  # サブサンプルサイズを2に設定（コメントアウト）
    params['fold'] = -1  # フォールド設定を-1に設定

# エポック数とLightGBMのモデル数を設定
params['n_epochs'] = 1  # エポック数を1に設定
params['n_lgb'] = 1  # 使用するLightGBMのモデル数を1に設定
params['model'] = 'microsoft/deberta-v3-small'  # 使用するモデルを指定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# params = {}
FULL = params.get('fold', 0) < 0
N_FOLDS = int(params.get('n_folds', 3)); 
FOLD = int(params.get('fold', 0))
SEED = int(params.get('seed', 3))
SS = int(params.get('subsample', 1))

print(N_FOLDS, FOLD, SEED, SS)
```

</div>
<div class="column-right">

# 日本語訳

```python
# パラメータの設定を確認
FULL = params.get('fold', 0) < 0  # フォールドの値が0未満ならFULLをTrueに設定
N_FOLDS = int(params.get('n_folds', 3))  # フォールド数を取得（デフォルトは3）
FOLD = int(params.get('fold', 0))  # 現在のフォールドを取得（デフォルトは0）
SEED = int(params.get('seed', 3))  # シード値を取得（デフォルトは3）
SS = int(params.get('subsample', 1))  # サブサンプルサイズを取得（デフォルトは1）

# 設定したパラメータを出力
print(N_FOLDS, FOLD, SEED, SS)
```

</div>
</details>

In [ ]:
# パラメータの設定を確認
FULL = params.get('fold', 0) < 0  # フォールドの値が0未満ならFULLをTrueに設定
N_FOLDS = int(params.get('n_folds', 3))  # フォールド数を取得（デフォルトは3）
FOLD = int(params.get('fold', 0))  # 現在のフォールドを取得（デフォルトは0）
SEED = int(params.get('seed', 3))  # シード値を取得（デフォルトは3）
SS = int(params.get('subsample', 1))  # サブサンプルサイズを取得（デフォルトは1）

# 設定したパラメータを出力
print(N_FOLDS, FOLD, SEED, SS)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from sklearn.model_selection import StratifiedKFold

def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state = SEED, shuffle = True)\
                    .split(X = np.zeros(len(train)), y = train.iloc[:, -3:].idxmax(1)))

train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
if SS > 1: train_ids, test_ids = train_ids[::SS], test_ids[::SS]

print(len(train_ids), len(test_ids));  assert set(train_ids) & set(test_ids) == set() 
```

</div>
<div class="column-right">

# 日本語訳

```python
from sklearn.model_selection import StratifiedKFold  # StratifiedKFoldをインポート

# フォールドを生成する関数を定義
def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state=SEED, shuffle=True)\
                    .split(X=np.zeros(len(train)), y=train.iloc[:, -3:].idxmax(1)))  # StratifiedKFoldで分割

# 現在のフォールドに基づいてトレーニングIDとテストIDを取得
train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
# サブサンプルサイズが1より大きい場合、インデックスをサンプリング
if SS > 1: 
    train_ids, test_ids = train_ids[::SS], test_ids[::SS]

# トレーニングIDとテストIDの長さを出力し、重複がないことを確認
print(len(train_ids), len(test_ids));  
assert set(train_ids) & set(test_ids) == set()  # トレーニングIDとテストIDに重複がないことを確認
```

</div>
</details>

In [ ]:
from sklearn.model_selection import StratifiedKFold  # StratifiedKFoldをインポート

# フォールドを生成する関数を定義
def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state=SEED, shuffle=True)\
                    .split(X=np.zeros(len(train)), y=train.iloc[:, -3:].idxmax(1)))  # StratifiedKFoldで分割

# 現在のフォールドに基づいてトレーニングIDとテストIDを取得
train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
# サブサンプルサイズが1より大きい場合、インデックスをサンプリング
if SS > 1: 
    train_ids, test_ids = train_ids[::SS], test_ids[::SS]

# トレーニングIDとテストIDの長さを出力し、重複がないことを確認
print(len(train_ids), len(test_ids));  
assert set(train_ids) & set(test_ids) == set()  # トレーニングIDとテストIDに重複がないことを確認

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def join_strings(x, ):
    x = ' '.join(['' if e is None else e for e in x]) if isinstance(x, list) else x
    return x
```

</div>
<div class="column-right">

# 日本語訳

```python
# リスト内の文字列を結合する関数を定義
def join_strings(x):
    # xがリストの場合、要素を結合して1つの文字列にする
    x = ' '.join(['' if e is None else e for e in x]) if isinstance(x, list) else x
    return x  # 結合された文字列またはそのままのxを返す
```

</div>
</details>

In [ ]:
# リスト内の文字列を結合する関数を定義
def join_strings(x):
    # xがリストの場合、要素を結合して1つの文字列にする
    x = ' '.join(['' if e is None else e for e in x]) if isinstance(x, list) else x
    return x  # 結合された文字列またはそのままのxを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def len_join_strings(x, ):
    return len(join_strings(x).split())
```

</div>
<div class="column-right">

# 日本語訳

```python
# 結合された文字列の単語数を返す関数を定義
def len_join_strings(x):
    # join_strings関数を使って文字列を結合し、単語数をカウントして返す
    return len(join_strings(x).split())  # 結合された文字列を空白で分割し、その長さを返す
```

</div>
</details>

In [ ]:
# 結合された文字列の単語数を返す関数を定義
def len_join_strings(x):
    # join_strings関数を使って文字列を結合し、単語数をカウントして返す
    return len(join_strings(x).split())  # 結合された文字列を空白で分割し、その長さを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def len_join_strings_j(x):
    x = json.loads(x)
    return len_join_strings(x)
```

</div>
<div class="column-right">

# 日本語訳

```python
# JSON形式の文字列を読み込み、結合された文字列の単語数を返す関数を定義
def len_join_strings_j(x):
    # JSON文字列をPythonオブジェクトに変換
    x = json.loads(x)
    # len_join_strings関数を使用して単語数を返す
    return len_join_strings(x)  # 結合された文字列の単語数を返す
```

</div>
</details>

In [ ]:
# JSON形式の文字列を読み込み、結合された文字列の単語数を返す関数を定義
def len_join_strings_j(x):
    # JSON文字列をPythonオブジェクトに変換
    x = json.loads(x)
    # len_join_strings関数を使用して単語数を返す
    return len_join_strings(x)  # 結合された文字列の単語数を返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
torch.manual_seed(datetime.datetime.now().microsecond)
random.seed(datetime.datetime.now().microsecond)
np.random.seed(datetime.datetime.now().microsecond)
```

</div>
<div class="column-right">

# 日本語訳

```python
# ランダムシードを設定
torch.manual_seed(datetime.datetime.now().microsecond)  # PyTorchのシードを設定
random.seed(datetime.datetime.now().microsecond)  # Pythonのrandomモジュールのシードを設定
np.random.seed(datetime.datetime.now().microsecond)  # NumPyのシードを設定
```

</div>
</details>

In [ ]:
# ランダムシードを設定
torch.manual_seed(datetime.datetime.now().microsecond)  # PyTorchのシードを設定
random.seed(datetime.datetime.now().microsecond)  # Pythonのrandomモジュールのシードを設定
np.random.seed(datetime.datetime.now().microsecond)  # NumPyのシードを設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# TRAIN = True and not KAGGLE
TRAIN = False
INFER = True # or KAGGLE 
SAVE = False
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニング、推論、保存のフラグを設定
# TRAIN = True and not KAGGLE  # 学習を行うかどうかのフラグ（Kaggleでは行わない場合はTrue）
TRAIN = False  # トレーニングを行わない設定
INFER = True  # 推論を行う設定（Kaggleでは行う場合もTrue）
SAVE = False  # 保存を行わない設定
```

</div>
</details>

In [ ]:
# トレーニング、推論、保存のフラグを設定
# TRAIN = True and not KAGGLE  # 学習を行うかどうかのフラグ（Kaggleでは行わない場合はTrue）
TRAIN = False  # トレーニングを行わない設定
INFER = True  # 推論を行う設定（Kaggleでは行う場合もTrue）
SAVE = False  # 保存を行わない設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
```

</div>
<div class="column-right">

# 日本語訳

```python
# LightGBMライブラリとテキストのカウントベクタイザーをインポート
import lightgbm as lgb  # LightGBMをインポート
from sklearn.feature_extraction.text import CountVectorizer  # CountVectorizerをインポート
```

</div>
</details>

In [ ]:
# LightGBMライブラリとテキストのカウントベクタイザーをインポート
import lightgbm as lgb  # LightGBMをインポート
from sklearn.feature_extraction.text import CountVectorizer  # CountVectorizerをインポート

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
LGB = True
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0
INFER_LGB = not TRAIN and LGB
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))
```

</div>
<div class="column-right">

# 日本語訳

```python
# LightGBMのトレーニングおよび推論フラグを設定
LGB = True  # LightGBMを使用する設定
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0  # トレーニングの条件を満たすかどうか
INFER_LGB = not TRAIN and LGB  # 推論を行う条件を設定

# 保存されたカウントベクタイザーを読み込む
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))  # 通常のカウントベクタイザーを読み込む
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))  # クリーンカウントベクタイザーを読み込む
```

</div>
</details>

In [ ]:
# LightGBMのトレーニングおよび推論フラグを設定
LGB = True  # LightGBMを使用する設定
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0  # トレーニングの条件を満たすかどうか
INFER_LGB = not TRAIN and LGB  # 推論を行う条件を設定

# 保存されたカウントベクタイザーを読み込む
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))  # 通常のカウントベクタイザーを読み込む
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))  # クリーンカウントベクタイザーを読み込む

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def symlog(x): return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)

def dense(x):
    x = np.asarray(x.astype(np.float32).todense())
    x = symlog(x)
    return x

def get_features(df):
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    v = np.hstack([
    # pfeat, 
          afeat - bfeat, np.abs(afeat - bfeat), 
    # afeat + bfeat
        ])
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)
    except: pass

    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)
            
    extras.append(df[c].str.len())
    extras.append(df[c].str.split().apply(lambda x: len(x)))
    
    extras = np.stack(extras, axis = 1)
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])
    return np.hstack([v, extras])
    # return v

```

</div>
<div class="column-right">

# 日本語訳

```python
# シンログ変換を行う関数を定義
def symlog(x): 
    return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)  # シンログ変換を適用

# 密な行列に変換してシンログ変換を行う関数を定義
def dense(x):
    x = np.asarray(x.astype(np.float32).todense())  # 密な行列に変換
    x = symlog(x)  # シンログ変換を適用
    return x  # 変換後の行列を返す

# 特徴量を取得するための関数を定義
def get_features(df):
    # プロンプトの特徴量を取得
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])
    
    # 応答Aの特徴量を取得
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    
    # 応答Bの特徴量を取得
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    # 特徴量を結合
    v = np.hstack([
          # pfeat,  # プロンプトの特徴量を含める場合のコメント
          afeat - bfeat,  # 応答Aと応答Bの差分
          np.abs(afeat - bfeat),  # 応答Aと応答Bの絶対差
          # afeat + bfeat  # 応答Aと応答Bの和を含める場合のコメント
        ])
    
    # 全投票モデルの数で正規化
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)  # データフレームの長さに基づいて正規化
    except: 
        pass  # エラーが発生した場合は無視

    # 追加の特徴量を取得
    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)  # 各文字列の出現回数をカウント
            
    # 文字列の長さと単語数を追加
    extras.append(df[c].str.len())  # 文字列の長さ
    extras.append(df[c].str.split().apply(lambda x: len(x)))  # 単語数
    
    # 追加の特徴量をスタックして変換
    extras = np.stack(extras, axis=1)  # 新たに追加された特徴量を結合
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])  # ルートとログ変換を適用
    
    # 結合した特徴量を返す
    return np.hstack([v, extras])  # 基本特徴量と追加特徴量を結合して返す
    # return v  # 基本特徴量のみを返す場合のコメント
```

</div>
</details>

In [ ]:
# シンログ変換を行う関数を定義
def symlog(x): 
    return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)  # シンログ変換を適用

# 密な行列に変換してシンログ変換を行う関数を定義
def dense(x):
    x = np.asarray(x.astype(np.float32).todense())  # 密な行列に変換
    x = symlog(x)  # シンログ変換を適用
    return x  # 変換後の行列を返す

# 特徴量を取得するための関数を定義
def get_features(df):
    # プロンプトの特徴量を取得
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])
    
    # 応答Aの特徴量を取得
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    
    # 応答Bの特徴量を取得
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    # 特徴量を結合
    v = np.hstack([
          # pfeat,  # プロンプトの特徴量を含める場合のコメント
          afeat - bfeat,  # 応答Aと応答Bの差分
          np.abs(afeat - bfeat),  # 応答Aと応答Bの絶対差
          # afeat + bfeat  # 応答Aと応答Bの和を含める場合のコメント
        ])
    
    # 全投票モデルの数で正規化
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)  # データフレームの長さに基づいて正規化
    except: 
        pass  # エラーが発生した場合は無視

    # 追加の特徴量を取得
    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)  # 各文字列の出現回数をカウント
            
    # 文字列の長さと単語数を追加
    extras.append(df[c].str.len())  # 文字列の長さ
    extras.append(df[c].str.split().apply(lambda x: len(x)))  # 単語数
    
    # 追加の特徴量をスタックして変換
    extras = np.stack(extras, axis=1)  # 新たに追加された特徴量を結合
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])  # ルートとログ変換を適用
    
    # 結合した特徴量を返す
    return np.hstack([v, extras])  # 基本特徴量と追加特徴量を結合して返す
    # return v  # 基本特徴量のみを返す場合のコメント

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 保存されたLightGBMモデルを読み込む
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))  # LightGBMモデルを読み込んで変数に格納
```

</div>
</details>

In [ ]:
# 保存されたLightGBMモデルを読み込む
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))  # LightGBMモデルを読み込んで変数に格納

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
if INFER and params.get('n_lgb', 1) > 0:
    df = test
    yps = []; b = 1000
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])
        ypms = []
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))
        yps.append(np.stack(ypms).mean(0))
        # break;
        print('.', end = '')
        
        if len(yps) % 2 == 0:
            gc.collect()
    print()

    yp = np.concatenate(yps)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 推論を行う条件が満たされている場合
if INFER and params.get('n_lgb', 1) > 0:
    df = test  # テストデータをdfに設定
    yps = []  # 予測結果を格納するリストを初期化
    b = 1000  # バッチサイズを設定
    for i in range(0, len(df), b):  # バッチ処理でデータを処理
        arr = get_features(df.iloc[i: i + b])  # 特徴量を取得
        ypms = []  # 各モデルの予測値を格納するリストを初期化
        for model in lgb_models:  # 各LightGBMモデルに対して予測
            ypms.append(model.predict_proba(arr))  # 予測確率を取得
            
        yps.append(np.stack(ypms).mean(0))  # モデルの予測値を平均してリストに追加
        # break;  # ループを中断する場合のコメント
        print('.', end = '')  # 進行状況を表示
        
        # 指定した条件ごとにガベージコレクションを実行
        if len(yps) % 2 == 0:  
            gc.collect()  # メモリを解放
    print()

    # すべての予測結果を連結
    yp = np.concatenate(yps)  # 予測値を一つの配列にまとめる
```

</div>
</details>

In [ ]:
# 推論を行う条件が満たされている場合
if INFER and params.get('n_lgb', 1) > 0:
    df = test  # テストデータをdfに設定
    yps = []  # 予測結果を格納するリストを初期化
    b = 1000  # バッチサイズを設定
    for i in range(0, len(df), b):  # バッチ処理でデータを処理
        arr = get_features(df.iloc[i: i + b])  # 特徴量を取得
        ypms = []  # 各モデルの予測値を格納するリストを初期化
        for model in lgb_models:  # 各LightGBMモデルに対して予測
            ypms.append(model.predict_proba(arr))  # 予測確率を取得
            
        yps.append(np.stack(ypms).mean(0))  # モデルの予測値を平均してリストに追加
        # break;  # ループを中断する場合のコメント
        print('.', end = '')  # 進行状況を表示
        
        # 指定した条件ごとにガベージコレクションを実行
        if len(yps) % 2 == 0:  
            gc.collect()  # メモリを解放
    print()

    # すべての予測結果を連結
    yp = np.concatenate(yps)  # 予測値を一つの配列にまとめる

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_preds = yp
```

</div>
<div class="column-right">

# 日本語訳

```python
# LightGBMモデルによる予測結果を変数に格納
lgb_preds = yp  # 予測値をlgb_predsに設定
```

</div>
</details>

In [ ]:
# LightGBMモデルによる予測結果を変数に格納
lgb_preds = yp  # 予測値をlgb_predsに設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_wt = 0.2 
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds
```

</div>
<div class="column-right">

# 日本語訳

```python
# LightGBMの予測とLLamaの予測を組み合わせて最終予測を計算
lgb_wt = 0.2  # LightGBMの重みを設定
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds  # 重み付き平均を計算して最終予測を生成
```

</div>
</details>

In [ ]:
# LightGBMの予測とLLamaの予測を組み合わせて最終予測を計算
lgb_wt = 0.2  # LightGBMの重みを設定
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds  # 重み付き平均を計算して最終予測を生成

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
out = pd.DataFrame(preds, 
                index = df.id, 
                    columns = train.columns[-3:])
display(out.head())
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最終予測結果をデータフレームに変換
out = pd.DataFrame(preds, 
                index=df.id,  # データフレームのインデックスをテストデータのIDに設定
                columns=train.columns[-3:])  # 列名をトレーニングデータの最後の3列に設定

# 最初の5行を表示
display(out.head())  # 予測結果データフレームの最初の5行を表示
```

</div>
</details>

In [ ]:
# 最終予測結果をデータフレームに変換
out = pd.DataFrame(preds, 
                index=df.id,  # データフレームのインデックスをテストデータのIDに設定
                columns=train.columns[-3:])  # 列名をトレーニングデータの最後の3列に設定

# 最初の5行を表示
display(out.head())  # 予測結果データフレームの最初の5行を表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Part 2 (Gemma 2 4b QLora)

</div>
<div class="column-right">

# 日本語訳

# パート 2 (Gemma 2 4b QLora)

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
assert torch.cuda.device_count() == 2
```

</div>
<div class="column-right">

# 日本語訳

```python
# 使用可能なCUDAデバイスの数が2であることを確認
assert torch.cuda.device_count() == 2  # CUDAデバイスが2つであることを確認するアサーション
```

</div>
</details>

In [ ]:
# 使用可能なCUDAデバイスの数が2であることを確認
assert torch.cuda.device_count() == 2  # CUDAデバイスが2つであることを確認するアサーション

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Configurations

</div>
<div class="column-right">

# 日本語訳

## 設定

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5748'
    max_length = 2048
    batch_size = 4
    device = torch.device("cuda")    
    tta = False 
    spread_max_length = False 

cfg = Config()
```

</div>
<div class="column-right">

# 日本語訳

```python
# 設定を格納するためのデータクラスを定義
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'  # Gemmaモデルのディレクトリ
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5748'  # LoRaモデルのチェックポイントディレクトリ
    max_length = 2048  # 最大入力長を2048に設定
    batch_size = 4  # バッチサイズを4に設定
    device = torch.device("cuda")  # 使用するデバイスをCUDAに設定
    tta = False  # テストタイムデータ拡張を無効にするフラグ
    spread_max_length = False  # 最大長の分散を無効にするフラグ

cfg = Config()  # 設定のインスタンスを作成
```

</div>
</details>

In [ ]:
# 設定を格納するためのデータクラスを定義
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'  # Gemmaモデルのディレクトリ
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5748'  # LoRaモデルのチェックポイントディレクトリ
    max_length = 2048  # 最大入力長を2048に設定
    batch_size = 4  # バッチサイズを4に設定
    device = torch.device("cuda")  # 使用するデバイスをCUDAに設定
    tta = False  # テストタイムデータ拡張を無効にするフラグ
    spread_max_length = False  # 最大長の分散を無効にするフラグ

cfg = Config()  # 設定のインスタンスを作成

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Load & pre-process Data 

</div>
<div class="column-right">

# 日本語訳

## データのロードと前処理

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータをCSVファイルから読み込む
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込む
```

</div>
</details>

In [ ]:
# テストデータをCSVファイルから読み込む
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込む

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))


test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

display(test.head(5))
```

</div>
<div class="column-right">

# 日本語訳

```python
# テキストを処理する関数を定義
def process_text(text: str) -> str:
    # テキストを評価し、空白で結合して返す
    return " ".join(eval(text, {"null": ""}))

# 各列に対してprocess_text関数を適用
test.loc[:, 'prompt'] = test['prompt'].apply(process_text)  # プロンプトの処理
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)  # 応答Aの処理
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)  # 応答Bの処理

# 最初の5行を表示
display(test.head(5))  # 処理後のテストデータの最初の5行を表示
```

</div>
</details>

In [ ]:
# テキストを処理する関数を定義
def process_text(text: str) -> str:
    # テキストを評価し、空白で結合して返す
    return " ".join(eval(text, {"null": ""}))

# 各列に対してprocess_text関数を適用
test.loc[:, 'prompt'] = test['prompt'].apply(process_text)  # プロンプトの処理
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)  # 応答Aの処理
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)  # 応答Bの処理

# 最初の5行を表示
display(test.head(5))  # 処理後のテストデータの最初の5行を表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Tokenize

</div>
<div class="column-right">

# 日本語訳

## トークナイズ

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    # TODO: change prompt
    prompt = ["<prompt>: " + p for p in prompt]
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    if spread_max_length:
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    return input_ids, attention_mask
```

</div>
<div class="column-right">

# 日本語訳

```python
# テキストをトークナイズする関数を定義
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    # プロンプトにプレフィックスを追加
    prompt = ["<prompt>: " + p for p in prompt]
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]  # 応答Aにプレフィックスを追加
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]  # 応答Bにプレフィックスを追加
    
    if spread_max_length:
        # 最大長を分散させてトークナイズ
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # 入力IDを結合
        attention_mask = [[1]* len(i) for i in input_ids]  # アテンションマスクを作成
    else:
        # 通常のトークナイズ
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # テキストを結合
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)  # トークナイズ
        input_ids = tokenized.input_ids  # 入力IDを取得
        attention_mask = tokenized.attention_mask  # アテンションマスクを取得
        
    return input_ids, attention_mask  # 入力IDとアテンションマスクを返す
```

</div>
</details>

In [ ]:
# テキストをトークナイズする関数を定義
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    # プロンプトにプレフィックスを追加
    prompt = ["<prompt>: " + p for p in prompt]
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]  # 応答Aにプレフィックスを追加
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]  # 応答Bにプレフィックスを追加
    
    if spread_max_length:
        # 最大長を分散させてトークナイズ
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # 入力IDを結合
        attention_mask = [[1]* len(i) for i in input_ids]  # アテンションマスクを作成
    else:
        # 通常のトークナイズ
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # テキストを結合
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)  # トークナイズ
        input_ids = tokenized.input_ids  # 入力IDを取得
        attention_mask = tokenized.attention_mask  # アテンションマスクを取得
        
    return input_ids, attention_mask  # 入力IDとアテンションマスクを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%time

tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True
tokenizer.padding_side = "right"

data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
data["length"] = data["input_ids"].apply(len)

aug_data = pd.DataFrame()
aug_data["id"] = test["id"]
# swap response_a & response_b
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイザーの初期化とデータ処理の計測
%%time

# Gemmaトークナイザーを指定したディレクトリからロード
tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True  # 終了トークンを追加
tokenizer.padding_side = "right"  # パディングを右側に設定

# データフレームを初期化
data = pd.DataFrame()
data["id"] = test["id"]  # テストデータのIDを設定
# トークナイズして入力IDとアテンションマスクを取得
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
data["length"] = data["input_ids"].apply(len)  # 入力IDの長さを取得

# データの拡張用データフレームを初期化
aug_data = pd.DataFrame()
aug_data["id"] = test["id"]  # 拡張データのIDを設定
# 応答Aと応答Bを入れ替えてトークナイズ
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)  # 入力IDの長さを取得
```

</div>
</details>

In [ ]:
# トークナイザーの初期化とデータ処理の計測
%%time

# Gemmaトークナイザーを指定したディレクトリからロード
tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True  # 終了トークンを追加
tokenizer.padding_side = "right"  # パディングを右側に設定

# データフレームを初期化
data = pd.DataFrame()
data["id"] = test["id"]  # テストデータのIDを設定
# トークナイズして入力IDとアテンションマスクを取得
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
data["length"] = data["input_ids"].apply(len)  # 入力IDの長さを取得

# データの拡張用データフレームを初期化
aug_data = pd.DataFrame()
aug_data["id"] = test["id"]  # 拡張データのIDを設定
# 応答Aと応答Bを入れ替えてトークナイズ
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)  # 入力IDの長さを取得

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
print(tokenizer.decode(data["input_ids"][0]))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最初の入力IDをデコードして人間が読める形式に変換し、表示
print(tokenizer.decode(data["input_ids"][0]))  # 最初の入力IDをデコードして出力
```

</div>
</details>

In [ ]:
# 最初の入力IDをデコードして人間が読める形式に変換し、表示
print(tokenizer.decode(data["input_ids"][0]))  # 最初の入力IDをデコードして出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
print(tokenizer.decode(aug_data["input_ids"][0]))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 拡張データの最初の入力IDをデコードして人間が読める形式に変換し、表示
print(tokenizer.decode(aug_data["input_ids"][0]))  # 拡張データの最初の入力IDをデコードして出力
```

</div>
</details>

In [ ]:
# 拡張データの最初の入力IDをデコードして人間が読める形式に変換し、表示
print(tokenizer.decode(aug_data["input_ids"][0]))  # 拡張データの最初の入力IDをデコードして出力

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Load model

</div>
<div class="column-right">

# 日本語訳

## モデルのロード

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Load base model on GPU 0
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False,
)

# Load base model on GPU 1
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,
    use_cache=False,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# GPU 0にベースモデルをロード
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,  # 指定したディレクトリからモデルをロード
    device_map=device_0,  # モデルをGPU 0に配置
    use_cache=False,  # キャッシュを無効にする
)

# GPU 1にベースモデルをロード
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,  # 指定したディレクトリからモデルをロード
    device_map=device_1,  # モデルをGPU 1に配置
    use_cache=False,  # キャッシュを無効にする
)
```

</div>
</details>

In [ ]:
# GPU 0にベースモデルをロード
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,  # 指定したディレクトリからモデルをロード
    device_map=device_0,  # モデルをGPU 0に配置
    use_cache=False,  # キャッシュを無効にする
)

# GPU 1にベースモデルをロード
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,  # 指定したディレクトリからモデルをロード
    device_map=device_1,  # モデルをGPU 1に配置
    use_cache=False,  # キャッシュを無効にする
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

#### Load LoRA adapter

</div>
<div class="column-right">

# 日本語訳

#### LoRAアダプターのロード

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)
```

</div>
<div class="column-right">

# 日本語訳

```python
# LoRAアダプターをモデルにロード
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)  # GPU 0のモデルにLoRAアダプターをロード
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)  # GPU 1のモデルにLoRAアダプターをロード
```

</div>
</details>

In [ ]:
# LoRAアダプターをモデルにロード
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)  # GPU 0のモデルにLoRAアダプターをロード
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)  # GPU 1のモデルにLoRAアダプターをロード

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Inference

</div>
<div class="column-right">

# 日本語訳

## 推論

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()
        
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df
```

</div>
<div class="column-right">

# 日本語訳

```python
# 推論を行う関数を定義
@torch.no_grad()  # 勾配計算を無効にするデコレーター
@torch.cuda.amp.autocast()  # 自動混合精度を有効にするデコレーター
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各モデルの勝利確率を格納するリストを初期化
    
    # データをバッチ処理で推論
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # 現在のバッチの終わりのインデックスを計算
        tmp = df.iloc[start_idx:end_idx]  # 現在のバッチを取得
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストに変換
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクをリストに変換
        
        # パディングを適用してテンソルを準備
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",  # 最も長いシーケンスに合わせてパディング
            pad_to_multiple_of=None,
            return_tensors="pt",  # PyTorchテンソルとして返す
        )
        
        # モデルによる推論を実行
        outputs = model(**inputs.to(device))  # 入力を指定したデバイスに移動
        proba = outputs.logits.softmax(-1).cpu()  # ロジットにソフトマックス関数を適用して確率を計算
        
        # 各勝利確率をリストに追加
        a_win.extend(proba[:, 0].tolist())  # モデルAの勝利確率を追加
        b_win.extend(proba[:, 1].tolist())  # モデルBの勝利確率を追加
        tie.extend(proba[:, 2].tolist())  # 引き分けの確率を追加
    
    # データフレームに結果を追加
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df  # 結果が追加されたデータフレームを返す
```

</div>
</details>

In [ ]:
# 推論を行う関数を定義
@torch.no_grad()  # 勾配計算を無効にするデコレーター
@torch.cuda.amp.autocast()  # 自動混合精度を有効にするデコレーター
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各モデルの勝利確率を格納するリストを初期化
    
    # データをバッチ処理で推論
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # 現在のバッチの終わりのインデックスを計算
        tmp = df.iloc[start_idx:end_idx]  # 現在のバッチを取得
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストに変換
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクをリストに変換
        
        # パディングを適用してテンソルを準備
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",  # 最も長いシーケンスに合わせてパディング
            pad_to_multiple_of=None,
            return_tensors="pt",  # PyTorchテンソルとして返す
        )
        
        # モデルによる推論を実行
        outputs = model(**inputs.to(device))  # 入力を指定したデバイスに移動
        proba = outputs.logits.softmax(-1).cpu()  # ロジットにソフトマックス関数を適用して確率を計算
        
        # 各勝利確率をリストに追加
        a_win.extend(proba[:, 0].tolist())  # モデルAの勝利確率を追加
        b_win.extend(proba[:, 1].tolist())  # モデルBの勝利確率を追加
        tie.extend(proba[:, 2].tolist())  # 引き分けの確率を追加
    
    # データフレームに結果を追加
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df  # 結果が追加されたデータフレームを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

# sort by input length to fully leverage dynaminc padding
data = data.sort_values("length", ascending=False)
# the total #tokens in sub_1 and sub_2 should be more or less the same
sub_1 = data.iloc[0::2].copy()
sub_2 = data.iloc[1::2].copy()

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

print(f"elapsed time: {time.time() - st}")
```

</div>
<div class="column-right">

# 日本語訳

```python
# 計測開始時刻を記録
st = time.time()

# 入力の長さでソートして動的パディングを最大限活用
data = data.sort_values("length", ascending=False)
# サブセット1とサブセット2のトークン数がほぼ同じになるようにする
sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータをサブセット1にコピー
sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータをサブセット2にコピー

# スレッドプールを使用して同時に推論を実行
with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 並列処理を実行

# 結果をデータフレームとして結合
result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 各モデルの勝利確率を取得

# 経過時間を表示
print(f"経過時間: {time.time() - st}")
```

</div>
</details>

In [ ]:
# 計測開始時刻を記録
st = time.time()

# 入力の長さでソートして動的パディングを最大限活用
data = data.sort_values("length", ascending=False)
# サブセット1とサブセット2のトークン数がほぼ同じになるようにする
sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータをサブセット1にコピー
sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータをサブセット2にコピー

# スレッドプールを使用して同時に推論を実行
with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 並列処理を実行

# 結果をデータフレームとして結合
result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 各モデルの勝利確率を取得

# 経過時間を表示
print(f"経過時間: {time.time() - st}")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

if cfg.tta:
    data = aug_data.sort_values("length", ascending=False)  # sort by input length to boost speed
    sub_1 = data.iloc[0::2].copy()
    sub_2 = data.iloc[1::2].copy()

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    tta_result_df = pd.concat(list(results), axis=0)
    # recall TTA's order is flipped
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # average original result and TTA result.
    proba = (proba + tta_proba) / 2

print(f"elapsed time: {time.time() - st}")
```

</div>
<div class="column-right">

# 日本語訳

```python
# 計測開始時刻を記録
st = time.time()

# テストタイムデータ拡張（TTA）が有効な場合
if cfg.tta:
    data = aug_data.sort_values("length", ascending=False)  # 入力の長さでソートして速度を向上
    sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータをサブセット1にコピー
    sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータをサブセット2にコピー

    # スレッドプールを使用して同時に推論を実行
    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 並列処理を実行

    tta_result_df = pd.concat(list(results), axis=0)  # TTAの結果をデータフレームとして結合
    # TTAの順序を反転する
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values  
    # 元の結果とTTA結果を平均化
    proba = (proba + tta_proba) / 2  # 平均を取る

# 経過時間を表示
print(f"経過時間: {time.time() - st}")
```

</div>
</details>

In [ ]:
# 計測開始時刻を記録
st = time.time()

# テストタイムデータ拡張（TTA）が有効な場合
if cfg.tta:
    data = aug_data.sort_values("length", ascending=False)  # 入力の長さでソートして速度を向上
    sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータをサブセット1にコピー
    sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータをサブセット2にコピー

    # スレッドプールを使用して同時に推論を実行
    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 並列処理を実行

    tta_result_df = pd.concat(list(results), axis=0)  # TTAの結果をデータフレームとして結合
    # TTAの順序を反転する
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values  
    # 元の結果とTTA結果を平均化
    proba = (proba + tta_proba) / 2  # 平均を取る

# 経過時間を表示
print(f"経過時間: {time.time() - st}")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
result_df.loc[:, "winner_model_a"] = proba[:, 0]
result_df.loc[:, "winner_model_b"] = proba[:, 1]
result_df.loc[:, "winner_tie"] = proba[:, 2]
```

</div>
<div class="column-right">

# 日本語訳

```python
# データフレームに勝利確率を設定
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝利確率を追加
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝利確率を追加
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 引き分けの確率を追加
```

</div>
</details>

In [ ]:
# データフレームに勝利確率を設定
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝利確率を追加
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝利確率を追加
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 引き分けの確率を追加

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
result_df
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最終的な結果データフレームを表示
result_df  # 予測結果を含むデータフレームを出力
```

</div>
</details>

In [ ]:
# 最終的な結果データフレームを表示
result_df  # 予測結果を含むデータフレームを出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
out
```

</div>
<div class="column-right">

# 日本語訳

```python
# 予測結果データフレームを表示
out  # 最初の出力結果を含むデータフレームを出力
```

</div>
</details>

In [ ]:
# 予測結果データフレームを表示
out  # 最初の出力結果を含むデータフレームを出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
merged_df = pd.merge(out, result_df, on='id')
```

</div>
<div class="column-right">

# 日本語訳

```python
# 予測結果データフレームを結合
merged_df = pd.merge(out, result_df, on='id')  # ID列を基にoutデータフレームとresult_dfを結合
```

</div>
</details>

In [ ]:
# 予測結果データフレームを結合
merged_df = pd.merge(out, result_df, on='id')  # ID列を基にoutデータフレームとresult_dfを結合

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
merged_df
```

</div>
<div class="column-right">

# 日本語訳

```python
# 結合したデータフレームを表示
merged_df  # 結合結果を含むデータフレームを出力
```

</div>
</details>

In [ ]:
# 結合したデータフレームを表示
merged_df  # 結合結果を含むデータフレームを出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
merged_df['winner_model_a']= (0.7*merged_df['winner_model_a_y']) + (0.3*merged_df['winner_model_a_x'])
merged_df['winner_model_b']= (0.7*merged_df['winner_model_b_y']) + (0.3*merged_df['winner_model_b_x'])
merged_df['winner_tie']= (0.7*merged_df['winner_tie_y']) + (0.3*merged_df['winner_tie_x'])
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルA、モデルB、および引き分けの勝利確率を加重平均して計算
merged_df['winner_model_a'] = (0.7 * merged_df['winner_model_a_y']) + (0.3 * merged_df['winner_model_a_x'])  # モデルAの加重平均
merged_df['winner_model_b'] = (0.7 * merged_df['winner_model_b_y']) + (0.3 * merged_df['winner_model_b_x'])  # モデルBの加重平均
merged_df['winner_tie'] = (0.7 * merged_df['winner_tie_y']) + (0.3 * merged_df['winner_tie_x'])  # 引き分けの加重平均
```

</div>
</details>

In [ ]:
# モデルA、モデルB、および引き分けの勝利確率を加重平均して計算
merged_df['winner_model_a'] = (0.7 * merged_df['winner_model_a_y']) + (0.3 * merged_df['winner_model_a_x'])  # モデルAの加重平均
merged_df['winner_model_b'] = (0.7 * merged_df['winner_model_b_y']) + (0.3 * merged_df['winner_model_b_x'])  # モデルBの加重平均
merged_df['winner_tie'] = (0.7 * merged_df['winner_tie_y']) + (0.3 * merged_df['winner_tie_x'])  # 引き分けの加重平均

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
submission_df = merged_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 提出用データフレームを作成
submission_df = merged_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]  # 必要な列を選択

# CSVファイルとして保存
submission_df.to_csv('submission.csv', index=False)  # インデックスなしでCSVに保存

# 提出データフレームを表示
display(submission_df)  # 提出用データを表示
```

</div>
</details>

In [ ]:
# 提出用データフレームを作成
submission_df = merged_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]  # 必要な列を選択

# CSVファイルとして保存
submission_df.to_csv('submission.csv', index=False)  # インデックスなしでCSVに保存

# 提出データフレームを表示
display(submission_df)  # 提出用データを表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

credits: https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora

</div>
<div class="column-right">

# 日本語訳

クレジット: [https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora)

</div>